In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.python.keras import optimizers, losses, activations, models
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.python.keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import f1_score, accuracy_score


In [2]:
df_train = pd.read_csv("C:/00ETH/ml4h/Project1/archive/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("C:/00ETH/ml4h/Project1/archive/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]


In [19]:
def identity_block(x,filter):
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv1D(filter, kernel_size=5, padding = 'same')(x)
    #x = tf.keras.layers.BatchNormalization(axis=2)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv1D(filter, kernel_size=3, padding = 'same')(x)
    #x = tf.keras.layers.BatchNormalization(axis=2)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [20]:
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv1D(filter, kernel_size=5, padding = 'same')(x)
    #x = tf.keras.layers.BatchNormalization(axis=1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv1D(filter, kernel_size=3, padding = 'same')(x)
    #x = tf.keras.layers.BatchNormalization(axis=1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv1D(filter, kernel_size=3, padding = 'same')(x_skip)
    #x_skip = tf.keras.layers.BatchNormalization(axis=1)(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [21]:
def get_model():
    nclass = 5
    inp = Input(shape=(187, 1))
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(inp)
    img_1 = tf.keras.layers.Convolution1D(16, kernel_size=5, activation=activations.relu, padding="same")(img_1)
    img_1 = tf.keras.layers.MaxPool1D(pool_size=2)(img_1)
    img_1 = tf.keras.layers.Dropout(rate=0.1)(img_1)
           
    img_1 = identity_block(img_1,16)
        
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="same")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="same")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    
    img_1 = convolutional_block(img_1,256)
    
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_1)
    img_1 = tf.keras.layers.Conv1D(256, kernel_size=3, padding = 'same')(img_1)
    img_1 = tf.keras.layers.Activation('relu')(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = tf.keras.layers.Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = tf.keras.layers.Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [22]:
model = get_model()
file_path = "baseline_cnn_mitbih.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 187, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_43 (Conv1D)              (None, 187, 16)      96          input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_44 (Conv1D)              (None, 187, 16)      1296        conv1d_43[0][0]                  
__________________________________________________________________________________________________
max_pooling1d_9 (MaxPooling1D)  (None, 93, 16)       0           conv1d_44[0][0]                  
____________________________________________________________________________________________

In [24]:
model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

pred_test = model.predict(X_test)
pred_test = np.argmax(pred_test, axis=-1)

Train on 78798 samples, validate on 8756 samples
Epoch 1/1000

Epoch 00001: val_acc improved from 0.97921 to 0.97967, saving model to baseline_cnn_mitbih.h5
78798/78798 - 82s - loss: 0.0892 - acc: 0.9757 - val_loss: 0.0698 - val_acc: 0.9797
Epoch 2/1000

Epoch 00002: val_acc improved from 0.97967 to 0.98298, saving model to baseline_cnn_mitbih.h5
78798/78798 - 85s - loss: 0.0834 - acc: 0.9771 - val_loss: 0.0647 - val_acc: 0.9830
Epoch 3/1000

Epoch 00003: val_acc did not improve from 0.98298
78798/78798 - 85s - loss: 0.0804 - acc: 0.9776 - val_loss: 0.0636 - val_acc: 0.9826
Epoch 4/1000

Epoch 00004: val_acc did not improve from 0.98298
78798/78798 - 85s - loss: 0.0770 - acc: 0.9786 - val_loss: 0.0662 - val_acc: 0.9824
Epoch 5/1000

Epoch 00005: val_acc improved from 0.98298 to 0.98515, saving model to baseline_cnn_mitbih.h5
78798/78798 - 85s - loss: 0.0732 - acc: 0.9794 - val_loss: 0.0541 - val_acc: 0.9852
Epoch 6/1000

Epoch 00006: val_acc did not improve from 0.98515
78798/78798 - 8

In [25]:
f1 = f1_score(Y_test, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.9258927990223744 
Test accuracy score : 0.9867988306230586 
